In [10]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf

In [11]:
def fetch_data(ticker, start_date, end_date, interval='1d'):
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date, interval=interval)
    return df

In [12]:
aapl_historical = fetch_data('AAPL', '2022-01-01', '2022-12-31')

In [13]:
aapl_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.156426,180.130505,175.038235,179.273575,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,179.884310,180.189647,176.427070,176.998352,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,176.909699,177.461277,172.014418,172.290207,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.103581,172.664498,169.059520,169.414108,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.290735,171.521942,168.458698,169.581558,86709100,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-23 00:00:00-05:00,129.700104,131.186127,128.432032,130.631348,63814900,0.0,0.0
2022-12-27 00:00:00-05:00,130.155848,130.185567,127.520629,128.818420,69007800,0.0,0.0
2022-12-28 00:00:00-05:00,128.461759,129.809087,124.697171,124.865585,85438400,0.0,0.0


In [15]:
def simple_moving_average(df, window):
    return df['Close'].rolling(window=window).mean()

def bollinger_bands(df, window=20, num_std=2):
    df['BB_MA'] = simple_moving_average(df, window)
    df['BB_Std'] = df['Close'].rolling(window=window).std()
    df['BB_Upper'] = df['BB_MA'] + (df['BB_Std'] * num_std)
    df['BB_Lower'] = df['BB_MA'] - (df['BB_Std'] * num_std)
    return df

def atr(df, window=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=window).mean()
    return df

def macd(df, fast=12, slow=26, signal=9):
    ema_fast = df['Close'].ewm(span=fast, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    return df

def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


In [24]:
def mean_reversion_strategy(df, ma_window=20, z_score_window=20, rsi_window=14, 
                            z_score_threshold=1.5, rsi_oversold=40, rsi_overbought=60,
                            bb_window=20, bb_std=2, atr_window=14, macd_fast=12, 
                            macd_slow=26, macd_signal=9, vol_window=20):
    df['MA'] = simple_moving_average(df, ma_window)
    df['Z Score'] = (df['Close'] - df['MA']) / df['Close'].rolling(window=z_score_window).std()
    
    df = bollinger_bands(df, window=bb_window, num_std=bb_std)
    df = atr(df, window=atr_window)
    df = macd(df, fast=macd_fast, slow=macd_slow, signal=macd_signal)
    df = rsi(df, window=rsi_window)
    
    df['Volume_SMA'] = df['Volume'].rolling(window=vol_window).mean()
    
    df['Long'] = (
        (df['Z Score'] < -z_score_threshold) & 
        (df['RSI'] < rsi_oversold) &
        ((df['Close'] < df['BB_Lower']) | (df['MACD'] > df['MACD_Signal']))
    )
    
    df['Short'] = (
        (df['Z Score'] > z_score_threshold) & 
        (df['RSI'] > rsi_overbought) &
        ((df['Close'] > df['BB_Upper']) | (df['MACD'] < df['MACD_Signal']))
    )
    
    df['Signal'] = np.where(df['Long'], 1, np.where(df['Short'], -1, 0))
    
    df['Position'] = df['Signal'].diff()
    return df

def backtest(df, initial_capital=10000):
    df['Return'] = df['Close'].pct_change()
    df['Strategy Return'] = df['Return'] * df['Signal'].shift(1)
    df['Equity'] = (1 + df['Strategy Return'].fillna(0)).cumprod() * initial_capital
    df['Drawdown'] = (df['Equity'] - df['Equity'].cummax()) / df['Equity'].cummax()
    
    total_return = (df['Equity'].iloc[-1] - initial_capital) / initial_capital
    strategy_return_std = df['Strategy Return'].std()
    if strategy_return_std != 0:
        sharpe_ratio = np.sqrt(252) * df['Strategy Return'].mean() / strategy_return_std
    else:
        sharpe_ratio = np.nan
    
    max_drawdown = df['Drawdown'].min()
    
    return df, {
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }

In [21]:
aapl_historical = mean_reversion_strategy(aapl_historical)
result_df, mets = backtest(aapl_historical)
print(mets)

{'Total Return': 0.0, 'Sharpe Ratio': nan, 'Max Drawdown': 0.0}


In [22]:
result_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return,MA,Z Score,...,MACD_Hist,RSI,Volume_SMA,Long,Short,Signal,Position,Strategy Return,Equity,Drawdown
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-23 00:00:00-05:00,129.700104,131.186127,128.432032,130.631348,63814900,0.0,0.0,-0.002798,139.350362,-1.519286,...,-1.096379,26.261582,81646890.0,False,False,0,0.0,-0.0,10000.0,0.0
2022-12-27 00:00:00-05:00,130.155848,130.185567,127.520629,128.818420,69007800,0.0,0.0,-0.013878,138.647472,-1.602974,...,-1.088441,27.960245,81634980.0,False,False,0,0.0,-0.0,10000.0,0.0
2022-12-28 00:00:00-05:00,128.461759,129.809087,124.697171,124.865585,85438400,0.0,0.0,-0.030685,137.898020,-1.902446,...,-1.253174,26.152303,81718710.0,False,False,0,0.0,-0.0,10000.0,0.0
2022-12-29 00:00:00-05:00,126.797421,129.264217,126.539849,128.402328,75703700,0.0,0.0,0.028324,136.985602,-1.255145,...,-1.036674,30.302150,79934850.0,False,False,0,0.0,0.0,10000.0,0.0
2022-12-30 00:00:00-05:00,127.213519,128.739163,126.242647,128.719345,77034200,0.0,0.0,0.002469,136.075163,-1.105348,...,-0.795686,31.430303,80224040.0,False,False,0,0.0,0.0,10000.0,0.0
